In [1]:
import org.apache.spark.sql.functions.{min, max}
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import scala.util.Random
import org.apache.spark.sql.functions.lit
import org.apache.spark.sql.Row
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions._
import scala.collection.mutable

## Variables

In [2]:
var trainFile = "../datasets/AOW-private/train_AOW_positive_only_cleaned.csv"
var testFile = "../datasets/AOW-private/test_AOW_positive_only_cleaned.csv"

trainFile = ../datasets/AOW-private/train_AOW_positive_only_cleaned.csv
testFile = ../datasets/AOW-private/test_AOW_positive_only_cleaned.csv


../datasets/AOW-private/test_AOW_positive_only_cleaned.csv

## Loading dataframes

In [3]:
var train = spark.read.options(Map("header" -> "true", "inferSchema" -> "true")).csv(trainFile)

train = [user: int, item: int ... 1 more field]


[user: int, item: int ... 1 more field]

In [4]:
var test = spark.read.options(Map("header" -> "true", "inferSchema" -> "true")).csv(testFile)

test = [user: int, item: int ... 1 more field]


[user: int, item: int ... 1 more field]

In [5]:
train.show(2)

+----+----+------+
|user|item|rating|
+----+----+------+
|   1| 241|     1|
|   1|1066|     1|
+----+----+------+
only showing top 2 rows



In [6]:
test.show(2)

+----+----+------+
|user|item|rating|
+----+----+------+
|   1| 375|     1|
|   1| 412|     1|
+----+----+------+
only showing top 2 rows



In [7]:
val model = new ALS()
    .setUserCol("user")
    .setItemCol("item")
    .setRatingCol("rating")
    .setPredictionCol("prediction")
    .setImplicitPrefs(true)
    .fit(train)


[Stage 349:==================================================>     (9 + 1) / 10]

model = als_5101f6787a4a


als_5101f6787a4a

In [8]:
model.setColdStartStrategy("drop")

als_5101f6787a4a

In [9]:
val trainPredictions = model.transform(train)
trainPredictions.show(5)

+-----+----+------+-----------+                                                 
| user|item|rating| prediction|
+-----+----+------+-----------+
| 1591| 148|     1|  0.0798986|
|17389| 148|     1| 0.11433779|
|17679| 148|     1|0.094594836|
|59990| 148|     1| 0.05463445|
|70863| 148|     1|0.029674664|
+-----+----+------+-----------+
only showing top 5 rows



trainPredictions = [user: int, item: int ... 2 more fields]


[user: int, item: int ... 2 more fields]

In [10]:
val testPredictions = model.transform(test)
testPredictions.show(5)

+------+----+------+-----------+                                                
|  user|item|rating| prediction|
+------+----+------+-----------+
|119813| 148|     1|0.069204345|
|155042| 148|     1|  0.0619019|
|183955| 148|     1| 0.06668936|
|184976| 148|     1| 0.06020306|
| 23709| 148|     1| 0.07009041|
+------+----+------+-----------+
only showing top 5 rows



testPredictions = [user: int, item: int ... 2 more fields]


[user: int, item: int ... 2 more fields]

In [11]:
val topUserRecommendations = model.recommendForAllUsers(1400).cache

topUserRecommendations = [user: int, recommendations: array<struct<item:int,rating:float>>]


[user: int, recommendations: array<struct<item:int,rating:float>>]

In [12]:
topUserRecommendations.show(5)

[Stage 432:=====================================================>(99 + 1) / 100]+----+--------------------+
|user|     recommendations|
+----+--------------------+
| 148|[[1477,0.8259419]...|
| 463|[[1477,0.55404705...|
| 471|[[1477,0.32402122...|
| 496|[[1477,0.79764533...|
| 833|[[1531,0.83137506...|
+----+--------------------+
only showing top 5 rows



In [13]:
val a_user_recommendations = topUserRecommendations.filter($"user" equalTo 148).cache

a_user_recommendations = [user: int, recommendations: array<struct<item:int,rating:float>>]


[user: int, recommendations: array<struct<item:int,rating:float>>]

In [35]:
val rec = a_user_recommendations.select("recommendations").collect().map { _.getList[Int](0) }

[Stage 641:====================================================>(198 + 1) / 200]

rec = Array([[1477,0.8259419], [217,0.66606104], [1775,0.6025304], [1023,0.5881221], [219,0.5687582], [1154,0.56452423], [1622,0.56200314], [1676,0.5278451], [1623,0.5089654], [1624,0.5001223], [1729,0.48991156], [858,0.45047283], [153,0.43833873], [1714,0.43433684], [1540,0.43121904], [1625,0.43108118], [741,0.42756113], [1689,0.42661756], [1244,0.4047393], [146,0.40396208], [1776,0.4012531], [1710,0.3987105], [1699,0.39391637], [1158,0.3817647], [1475,0.37630314], [1601,0.37228778], [1633,0.36384463], [625,0.36134726], [1258,0.35819983], [1639,0.3561262], [1310,0.3289353], [1117,0.32062143], [815,0.31764483], [1703,0.31176895], [1122,0.3111239], [669,0.30132324], [1657,0.30007738], [1495,0.29867092], [1396,0.2961182], [960,0.28893283], [1504,0.2815935], [22...


[[[1477,0.8259419], [217,0.66606104], [1775,0.6025304], [1023,0.5881221], [219,0.5687582], [1154,0.56452423], [1622,0.56200314], [1676,0.5278451], [1623,0.5089654], [1624,0.5001223], [1729,0.48991156], [858,0.45047283], [153,0.43833873], [1714,0.43433684], [1540,0.43121904], [1625,0.43108118], [741,0.42756113], [1689,0.42661756], [1244,0.4047393], [146,0.40396208], [1776,0.4012531], [1710,0.3987105], [1699,0.39391637], [1158,0.3817647], [1475,0.37630314], [1601,0.37228778], [1633,0.36384463], [625,0.36134726], [1258,0.35819983], [1639,0.3561262], [1310,0.3289353], [1117,0.32062143], [815,0.31764483], [1703,0.31176895], [1122,0.3111239], [669,0.30132324], [1657,0.30007738], [1495,0.29867092], [1396,0.2961182], [960,0.28893283], [1504,0.2815935], [225,0.28048828], [1454,0.27937633], [1743,0.27466395], [2,0.26742384], [1484,0.26225734], [977,0.25974447], [75,0.25556517], [1531,0.25373423], [1480,0.25082546], [1359,0.24918947], [1497,0.24860583], [35,0.2438609], [1336,0.24094039], [1053,0.

In [40]:
rec(0).size

1400

In [22]:
topUserRecommendations.orderBy($"user")

[Stage 511:====================================================>(199 + 1) / 200]+----+--------------------+
|user|     recommendations|
+----+--------------------+
|   1|[[1659,0.906803],...|
+----+--------------------+
only showing top 1 row



In [14]:
import topUserRecommendations.sparkSession.implicits._

In [15]:
val allUserItems = train.unionAll(test).cache

allUserItems = [user: int, item: int ... 1 more field]


[user: int, item: int ... 1 more field]

In [52]:
val allUserItemsSimple = allUserItems.select("user").dropDuplicates.crossJoin(allUserItems.select("item").dropDuplicates).cache

allUserItemsSimple = [user: int, item: int]


lastException: Throwable = null


[user: int, item: int]

In [53]:
val userItemsWithoutTrain = allUserItemsSimple.except(train.select("user", "item")).join(test, Array("user", "item")).cache

userItemsWithoutTrain = [user: int, item: int ... 1 more field]


[user: int, item: int ... 1 more field]

In [54]:
val predictions = model.transform(userItemsWithoutTrain).cache

predictions = [user: int, item: int ... 2 more fields]


[user: int, item: int ... 2 more fields]

In [55]:
predictions.count()

[Stage 795:====================================================>(199 + 1) / 200]10]

1577727

In [45]:
val users = mutable.Map[Int, Int]()
val userRecordSeen = udf { user: Int => {
    users.update(user, 1 + users.getOrElse(user, 0))
    users.get(user).get
}}

users = Map()
userRecordSeen = UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))


UserDefinedFunction(<function1>,IntegerType,Some(List(IntegerType)))

In [46]:
val predictionsWithSeenRecords = predictions.sort(asc("user"), desc("prediction")).withColumn("seen", userRecordSeen($"user")).cache

[Stage 726:==================================================>    (10 + 1) / 11]

predictionsWithSeenRecords = [user: int, item: int ... 2 more fields]


[user: int, item: int ... 2 more fields]

In [47]:
predictionsWithSeenRecords.show(100)

+----+----+--------------+----+                                                 
|user|item|    prediction|seen|
+----+----+--------------+----+
|   1| 399|    0.07074636|   1|
|   1|1479|   0.057538137|   2|
|   1| 514|   0.045268133|   3|
|   1|1645|   0.028614532|   4|
|   1| 643|   0.025234945|   5|
|   1|1470|    0.01788687|   6|
|   1| 215|   6.783702E-4|   7|
|   2|1427|    0.28039023|   1|
|   2|1668|    0.23149545|   2|
|   2|1631|    0.17042916|   3|
|   2|1363|    0.15161197|   4|
|   2|1179|    0.14710721|   5|
|   2| 632|   0.077703185|   6|
|   2| 850|   0.054404877|   7|
|   2| 232|   0.034221716|   8|
|   2| 891|   0.016462205|   9|
|   2| 399|  0.0076153325|  10|
|   3|1678|    0.11629292|   1|
|   3|1505|    0.11361963|   2|
|   3|1503|   0.052183814|   3|
|   3|1653|   0.035140984|   4|
|   3|1483|   0.029564083|   5|
|   3| 583|   0.023097873|   6|
|   3|1066|   0.022627495|   7|
|   3| 177|   0.021715939|   8|
|   3| 181|   0.018077735|   9|
|   3|1645|   0.0067881

In [114]:
def get_precision_at_k(predictions: Dataset[Row], k: Int) = {
    var precisions_sum: Long = 0
    
    users.map { user =>
        val topPredictions = predictions.filter($"user" equalTo user).orderBy(desc("prediction")).limit(k)
        topPredictions.filter($"rating" equalTo 1 ).count / k
    }.foreach { v: Long => precisions_sum += v }
    precisions_sum / predictions.count()
}

get_precision_at_k: (predictions: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row], k: Int)Long


In [115]:
get_precision_at_k(allPredictionsWithoutTrain, 5)

[Stage 92676:===============================================>   (186 + 1) / 200]

Name: org.apache.spark.SparkException
Message: Job 2830 cancelled as part of cancellation of all jobs
StackTrace:   at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1499)
  at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1439)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply$mcVI$sp(DAGScheduler.scala:723)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:723)
  at org.apache.spark.scheduler.DAGScheduler$$anonfun$doCancelAllJobs$1.apply(DAGScheduler.scala:723)
  at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
  at org.apache.spark.scheduler.DAGScheduler.doCancelAllJobs(DAGScheduler.scala:723)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:1692)
  at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1669

In [128]:
import scala.collection.mutable

watch_users = <function1>


<function1>